In [51]:
def gen_dct(fname):
    
    global dct
    dct = {}
    
    fh = open(fname)
    flines = fh.readlines()
    
    date_dct = {}
    
    for i in range(len(flines)):
        date = flines[i].split()[0][8:10]
        info = date_dct.get(date, [])
        info.append(flines[i].split())
        date_dct.update({date: info})
    
    for key, value in date_dct.items():
        for elm in value:
            hr1 = elm[1][0]
            hr2 = elm[1][1]
            mn1 = elm[1][3]
            mn2 = elm[1][4]
            sec1 = elm[1][6]
            sec2 = elm[1][7]
            
            time = ((int(hr1) * 10) + int(hr2)) + (((int(mn1) * 10) + int(mn2)) / 60) + (((int(sec1) * 10) + int(sec2)) / 3600)
                
            elm.append(str(time))
    
    for j in sorted(date_dct):
        dct.update({j: date_dct[j]})
        
    for k in dct.values():
        for item in k:
            k.sort(key = lambda item: item[5])
            
    print(dct)
            
    global s_dct
    s_dct = {}
    
    for key1, val1 in dct.items():
        for elm1 in val1:
            if elm1[2] == "login":
                var = elm1[0] + "|" + elm1[3] + "|" + elm1[4]
                s_lst = s_dct.get(var, [])
                s_lst.append(elm1)
                s_dct.update({var: s_lst})

In [52]:
gen_dct("userlog.log")

{'01': [['2020-05-01', '00:04:23', 'login', 'mailserver.local', 'lupe.gave@freesurf.fr', '0.07305555555555555'], ['2020-05-01', '00:08:47', 'logout', 'mailserver.local', 'kirsten.pflughoeft@mail2srilanka.com', '0.1463888888888889'], ['2020-05-01', '01:30:53', 'login', 'webserver.local', 'breena.benassi@gmx.net', '1.5147222222222223'], ['2020-05-01', '01:45:55', 'logout', 'webserver.local', 'maryelizabeth.ryba@freesurf.fr', '1.7652777777777777'], ['2020-05-01', '01:51:58', 'login', 'webserver.local', 'arti.karshner@mail2perry.com', '1.8661111111111113'], ['2020-05-01', '01:55:52', 'login', 'mailserver.local', 'daysha.karshner@blacksburg.net', '1.931111111111111'], ['2020-05-01', '10:00:13', 'login', 'mailserver.local', 'rayon.crumly@mail2champaign.com', '10.00361111111111'], ['2020-05-01', '10:09:52', 'logout', 'mailserver.local', 'tarrin.evanoff@blacksburg.net', '10.164444444444445'], ['2020-05-01', '10:17:09', 'logout', 'mailserver.local', 'laurinda.barrow@miho-nakayama.com', '10.2858

In [53]:
def sus_act():
                
    sus_dct = {}
        
    for s_key, s_val in s_dct.items():
        for var1 in range(len(s_val)):
            if float(s_val[var1][5]) < 6:
                sus_lst = sus_dct.get(s_key, [])
                sus_lst.append(s_val[var1])
                sus_dct.update({s_key: sus_lst})
    
    used_emails2 = []
    used_dates2 = []

    run2 = False
    
    saved_count = {}
    total = 0
            
    for sus_key2, sus_val2 in sus_dct.items():
        count = 0
        run2 = False
        email2 = sus_key2.split("local|")[1]
        if email2 not in used_emails2:
            used_emails2.append(email2)
            run2 = True
        if run2 == True:
            for sus_key3, sus_val3 in sus_dct.items():
                for y in sus_val3:
                    if email2 in sus_key3:
                        if y[0] not in used_dates2:
                            used_dates2.append(y[0])
                            count += 1
            used_dates2 = []
        saved_count[sus_key2] = count
        total += count
        
    sus_fh = open("sus_rep.txt", "w")
    sus_fh.write(f"===============================\n=== Suspicious Activities ({total} cases) ===\n===============================\n")
    
    used_emails = []
    used_dates = []
        
    for sus_key, sus_val in sus_dct.items():
        #print("Hello!")
        run = False
        email = sus_key.split("local|")[1]
        if email not in used_emails:
            sus_fh.write(f"{email}\t{saved_count[sus_key]}\n")
            used_emails.append(email)
            run = True
        if run == True:
            for sus_key1, sus_val1 in sus_dct.items():
                for l in sus_val1:
                    if email in sus_key1:
                        if l[0] not in used_dates:
                            sus_fh.write(f"\tDATE: [{l[0]}]\n")
                            used_dates.append(l[0])
                        sus_fh.write(f"\t\t{l[1]}\t{l[2]}\t{l[3]}\n")
            used_dates = []

In [54]:
sus_act()

In [55]:
def irregular_beh():
    
    irr_dct = {}
    sys_dct = {}
    
    in_dct = {}
    out_dct = {}
    
    for key2, val2 in dct.items():
        for elm2 in val2:
            if elm2[2] == "login":
                var1 = elm2[0] + "|" + elm2[3] + "|" + elm2[4]
                in_lst = in_dct.get(var1, [])
                in_lst.append(elm2)
                in_dct.update({var1: in_lst})
            if elm2[2] == "logout":
                var1 = elm2[0] + "|" + elm2[3] + "|" + elm2[4]
                out_lst = out_dct.get(var1, [])
                out_lst.append(elm2)
                out_dct.update({var1: out_lst})
                
    for in_key, in_val in in_dct.items():
        for out_key, out_val in out_dct.items():
            if in_key == out_key:
                if len(in_val) > len(out_val):
                    irr_dct.update({in_key: in_val + out_val})
                    irr_dct[in_key].sort(key = lambda item: item[5])
                elif len(in_val) < len(out_val):
                    sys_dct.update({in_key: in_val + out_val})
                    sys_dct[in_key].sort(key = lambda item: item[5])
                    
    irr_ct = len(irr_dct.keys())
                
    irr_fh = open("irr_rep.txt", "w")
    irr_fh.write(f"===============================\n=== Irresponsible Behavior ({irr_ct} cases) ===\n===============================\n")
    
    irr_used_emails = []
    irr_used_dates = []
    
    irr_run = False
    
    for irr_key, irr_val in irr_dct.items():
        irr_run = False
        irr_email = irr_key.split("local|")[1]
        if irr_email not in irr_used_emails:
            irr_fh.write(f"{irr_email}\n")
            irr_used_emails.append(irr_email)
            irr_run = True
        if irr_run == True:
            for irr_key2, irr_val2 in irr_dct.items():
                if irr_email in irr_key2:
                    for p in irr_val2:
                        if p[0] not in irr_used_dates:
                            irr_fh.write(f"\tDATE: [{p[0]}] ---\n")
                            irr_used_dates.append(p[0])
                        irr_fh.write(f"\t\t{p[1]}\t{p[2]}\t{p[3]}\n")
                    irr_used_dates = []
    
    sys_ct = len(sys_dct.keys())
                
    sys_fh = open("sys_rep.txt", "w")
    sys_fh.write(f"===============================\n=== System Glitches ({sys_ct} cases) ===\n===============================\n")
      
    sys_used_emails = []
    sys_used_dates = []
    
    sys_run = False
    
    for sys_key, sys_val in sys_dct.items():
        sys_run = False
        sys_email = sys_key.split("local|")[1]
        if sys_email not in sys_used_emails:
            sys_fh.write(f"{sys_email}\n")
            sys_used_emails.append(sys_email)
            sys_run = True
        if sys_run == True:
            for sys_key2, sys_val2 in sys_dct.items():
                if sys_email in sys_key2:
                    for q in sys_val2:
                        if q[0] not in sys_used_dates:
                            sys_fh.write(f"\tDATE: [{q[0]}] ---\n")
                            sys_used_dates.append(q[0])
                        sys_fh.write(f"\t\t{q[1]}\t{q[2]}\t{q[3]}\n")
                    sys_used_dates = []

In [56]:
irregular_beh()

In [57]:
def domain_ct():

    email_dct = {}
    
    for key3, val3 in dct.items():
        for elm3 in val3:
            t_email = elm3[4]
            email_dct[t_email] = email_dct.get(t_email, 0) + 1
            
    dom_fh = open("dom_rep.txt", "w")
    dom_fh.write(f"===============================\n=== Domain Count ({len(email_dct)} domains) ===\n===============================\n")
    
    for dom_key, dom_val in email_dct.items():
        dom_fh.write(f"{dom_key}\t{dom_val}\n")

In [ ]:
domain_ct()